# A full business solution

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [29]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [30]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [31]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [35]:
ed = Website("https://edwarddonner.com")
ed.links

['https://edwarddonner.com/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/18/2025-ai-executive-briefing/',
 'https://edwarddonner.com/2025/05/18/2025-ai-executive-briefing/',
 'https://edwarddonner.com/2025/04/21/the-complete-agentic-ai-engineering-course/',
 'https://edwarddonner.com/2025/04/21/the-

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [5]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}
"""

In [6]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}



In [7]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [8]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://edwarddonner.com - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
https://edwarddonner.com/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/
https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/
https://edwarddo

In [9]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [10]:
# Anthropic has made their site harder to scrape, so I'm using HuggingFace..

huggingface = Website("https://huggingface.co")
huggingface.links

['/',
 '/models',
 '/datasets',
 '/spaces',
 '/docs',
 '/enterprise',
 '/pricing',
 '/login',
 '/join',
 '/spaces',
 '/models',
 '/xai-org/grok-2',
 '/Qwen/Qwen-Image-Edit',
 '/microsoft/VibeVoice-1.5B',
 '/deepseek-ai/DeepSeek-V3.1',
 '/deepseek-ai/DeepSeek-V3.1-Base',
 '/models',
 '/spaces/enzostvs/deepsite',
 '/spaces/Qwen/Qwen-Image-Edit',
 '/spaces/zerogpu-aoti/wan2-2-fp8da-aoti-faster',
 '/spaces/importedFalcon2/face-search',
 '/spaces/lvwerra/jupyter-agent-2',
 '/spaces',
 '/datasets/fka/awesome-chatgpt-prompts',
 '/datasets/nvidia/Granary',
 '/datasets/nvidia/Llama-Nemotron-VLM-Dataset-v1',
 '/datasets/nvidia/Nemotron-Post-Training-Dataset-v2',
 '/datasets/liumindmind/NekoQA-10K',
 '/datasets',
 '/join',
 '/pricing#endpoints',
 '/pricing#spaces',
 '/pricing',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/allenai',
 '/facebook',
 '/amazon',
 '/google',
 '/Intel',
 '/microsoft',
 '/grammarly',
 '/Writer',
 '/doc

In [11]:
get_links("https://huggingface.co")

{'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'},
  {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'},
  {'type': 'blog page', 'url': 'https://huggingface.co/blog'},
  {'type': 'company page',
   'url': 'https://www.linkedin.com/company/huggingface/'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [12]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [13]:
print(get_all_details("https://huggingface.co"))

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'community discussions', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}]}
Landing page:
Webpage Title:
Hugging Face – The AI community building the future.
Webpage Contents:
Hugging Face
Models
Datasets
Spaces
Community
Docs
Enterprise
Pricing
Log In
Sign Up
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Explore AI Apps
or
Bro

In [25]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."


In [15]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [16]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/about'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'company page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'contact page', 'url': 'https://huggingface.co/chat'}, {'type': 'social media page', 'url': 'https://twitter.com/huggingface'}, {'type': 'social media page', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'community forum', 'url': 'https://discuss.huggingface.co'}, {'type': 'status page', 'url': 'https://status.huggingface.co/'}]}


"You are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nHugging Face – The AI community building the future.\nWebpage Contents:\nHugging Face\nModels\nDatasets\nSpaces\nCommunity\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 1M+ models\nTrending on\nthis week\nModels\nxai-org/grok-2\nUpdated\n4 days ago\n•\n3.27k\n•\n811\nmicrosoft/VibeVoice-1.5B\nUpdated\nabout 15 hours ago\n•\n10.2k\n•\n784\nQwen/Qwen-Image-Edit\nUpdated\n3 days ago\n•\n51.2k\n•\n1.46k\ndeepseek-ai/DeepSeek-V3.1\nUpdated\n2 days ago\n•\n38.2k\n•\n622\nopenbmb/MiniCPM-V-4_5\nUpdated\nabout 15 hours ago\n•\n1.46k\n•\n463\nBrowse 1M+ models\nSpaces\nRunning\n12.4k\n12.4k\nDeepSite v2\n🐳\

In [26]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [18]:
create_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/about'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'company page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'documentation page', 'url': 'https://huggingface.co/docs'}]}


```markdown
# Hugging Face - Your Partner in AI Innovation

## About Us
Hugging Face is a thriving community at the focal point of artificial intelligence, dedicated to building the future of machine learning. Our platform encourages collaboration among developers and researchers in creating and sharing models, datasets, and applications. Over **50,000 organizations** globally, including industry giants like **Google, Microsoft,** and **Amazon**, trust us to accelerate their AI projects.

## Our Offerings
- **Models**: Explore and contribute to over **1 million machine learning models**. Our repository includes trending models like **Grok-2** and **VibeVoice-1.5B**, constantly updated by our innovative community.
- **Datasets**: Access a vast collection of **250,000+ datasets** tailored for various ML tasks, fostering inclusive research and development.
- **Spaces**: Host and utilize applications effortlessly. Collaborate in real-time with community-built applications, running thousands of instances every day.

## Enterprise Solutions
Scale your organization with enterprise-grade security and support. Our paid compute solutions provide optimized inference endpoints and dedicated support to ensure you have the resources needed to innovate without limitations. 

- **Team & Enterprise Plans** start at **$20/user/month.** 

## Company Culture
At Hugging Face, we value **open-source collaboration** and a friendly community ethos. We pride ourselves on fostering an inclusive environment where diverse talents can thrive. Our culture encourages learning and sharing, empowering employees to push the boundaries of what is possible in AI. 

## Careers at Hugging Face
Join our innovative and passionate team! If you're enthusiastic about advancing the field of AI and ML, check our **[Current Openings](https://huggingface.co/jobs)**. We are looking for talented individuals ready to contribute to groundbreaking projects in machine learning and data science.

## Get Involved
Whether you are a researcher, developer, or a business looking to integrate AI, Hugging Face welcomes you. 
- **Start collaborating** by signing up for free at [Hugging Face](https://huggingface.co).
- Dive into our community and explore the endless possibilities in the world of machine learning!

---

**Follow Us**:  
- [Twitter](https://twitter.com/huggingface)  
- [LinkedIn](https://www.linkedin.com/company/huggingface)  
- [Discord](https://discord.gg/huggingface)

---

Together, let's build the future of AI!
```

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [27]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [20]:
stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'models page', 'url': 'https://huggingface.co/models'}, {'type': 'datasets page', 'url': 'https://huggingface.co/datasets'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}]}


# Hugging Face Brochure

## **Welcome to Hugging Face**
**The AI community building the future.**

Hugging Face is at the forefront of the AI revolution, providing a robust platform for collaboration within the machine learning community. Here, users can explore, develop, and share state-of-the-art machine learning models, datasets, and applications.

---

## **What We Offer**

### **Extensive Model Library**
- Over **1 million models** available.
- Collaborate on trending AI models updated frequently.
- Specialized resources for multiple modalities including text, image, and audio.

### **Diverse Datasets**
- Access **250k+ datasets** for various ML tasks and studies.
- Community-driven contributions for richer dataset availability.

### **Innovative Spaces & Applications**
- Host and create applications within **Spaces**, enhancing collaborative opportunities.
- Ranging from image editing tools to data analyzers running on optimized environments.

### **Enterprise Solutions**
For businesses, we offer dedicated support and enterprise-grade solutions, including:
- Optimized inference endpoints.
- Single Sign-On and access control.
- Competitive pricing starting at **$20/user/month**.

---

## **Community-Centric Culture**

At Hugging Face, collaboration is key. We empower our users to build and share their work while promoting a culture of transparency and inclusion. Over **50,000 organizations**, including leading enterprises like Google, Microsoft, and Amazon, utilize our platforms, recognizing the value of community-driven innovation.

### **Open Source Commitment**
Hugging Face is committed to open-source principles, continuously building the foundation of machine learning tooling with our community:
- **Transformers** library: Over **148,900** state-of-the-art AI models.
- **Diffusers** and **Tokenizers** to enhance the ML workflow.

---

## **Careers at Hugging Face**

Join a passionate team dedicated to shaping the future of AI! We are always on the lookout for innovative minds to help us expand our offerings and improve our community. Our ideal team members are curious, collaborative, and eager to make an impact in the AI landscape. 

### **Current Opportunities**
Explore and apply for exciting roles in engineering, product development, community management, and more on our [Jobs page](https://huggingface.co/jobs).

---

## **Connect with Us**

Join the conversation and discover the future of AI at Hugging Face. Follow us on our social platforms for the latest updates and community contributions.

- [GitHub](https://github.com/huggingface)
- [Twitter](https://twitter.com/huggingface)
- [LinkedIn](https://www.linkedin.com/company/hugging-face)
- [Discord](https://discord.com/invite/huggingface)

---

#### **Explore. Collaborate. Innovate. Join the Hugging Face community today!**

[Visit our website](https://huggingface.co) to learn more! 

In [28]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'company blog', 'url': 'https://huggingface.co/blog'}, {'type': 'company page', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'community discussion', 'url': 'https://discuss.huggingface.co'}]}



# Welcome to the Hugging Face Brochure! 🌟

🎉 **Join the AI Revolution!** 🎉  
At Hugging Face, we don't just dabble in AI; we’re the friendly giants lifting it to the sky! Think of us as the social hub for machine learning enthusiasts, where every byte counts, and every pixel has a purpose!

---

## Who We Are 🤔

**The AI Community Building the Future!**  
We're like the Avengers, but with more coded capes and less spandex. At Hugging Face, we empower researchers, developers, and curious minds to build, share, and collaborate on AI models and applications. With over **1 million models** and **250k datasets**, you'll never run out of things to explore! (No really, we’ve tried—it's impossible.)

---

## What We Offer 🌐

- **💡 Models Galore:**  
  From text to videos to AI companions (yes, we said it!), explore an endless selection of state-of-the-art models. 

- **📊 Datasets Galore:**  
  Tired of staring at spreadsheets? We've got over 250k datasets that could make even a data analyst swoon! 

- **🏗️ Spaces and Applications:**  
  Create your own **Hugging Space**—like a playground, but with more data and fewer swings!  

- **🚀 Enterprise Solutions:**  
  Is your company tired of slow AI? Our enterprise packages are designed to rocket your models into the upper echelons of computing efficiency! 

---

## Who Uses Us? 🤝

More than **50,000 organizations** trust Hugging Face, including AI giants like:
- Amazon (Yes, we whisper sweet nothings to Alexa)
- Microsoft (Helping Cortana to be just a little more charming)
- Google (Might help you with your next search strategy!)

🌟 Join the ranks of trendsetters and innovators who are shaping the future of machine learning!

---

## Culture & Careers 🎈

At Hugging Face, we believe in collaboration, positivity, and, of course, a bit of playful competition!  
- Passionate about AI?  
- Love working in a community-centric environment?  
- Thrilled by the idea of learning and growing every day?  

If you answered “yes” to any of these questions, check out our **career opportunities**! From engineers to community managers, we've got room for all sorts of innovators.  

**Bonus:** Bring your pet to Zoom meetings. (Just kidding—unless they can help with ML, then let’s talk.)

---

## Let’s Get Social! 📱

Find us on:
- **GitHub**: Where all the cool kids hang out
- **Twitter**: Catch our latest updates, memes, and AI dad jokes
- **LinkedIn**: Far less fun, but great for networking!

---

Remember, in the world of AI, it's not just about the algorithms; it's about the hugs (or Hugging Face). So why not give us a try? Join us today, and let’s build something amazing together!

**Ready to embrace the future? [Sign Up Now!](https://huggingface.co/signup)**  

📍 *Life is short, make it sweet (just like that Hugging Face model you’ve been eyeing).*

---


In [23]:
def stream_brochure2(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    for chunk in stream:
        print(chunk.choices[0].delta.content or '', end='')
    # response = ""
    # display_handle = display(Markdown(""), display_id=True)
    # for chunk in stream:
    #     response += chunk.choices[0].delta.content or ''
    #     response = response.replace("```","").replace("markdown", "")
    #     update_display(Markdown(response), display_id=display_handle.display_id)

In [24]:
stream_brochure2("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'documentation page', 'url': 'https://huggingface.co/docs'}, {'type': 'company profile on LinkedIn', 'url': 'https://www.linkedin.com/company/huggingface'}]}
# Hugging Face Brochure

---

## About Us

**Hugging Face** is at the forefront of the AI community, dedicated to **building the future** through collaboration and innovation. Our platform enables machine learning enthusiasts, researchers, and companies to create, discover, and collaborate on cutting-edge models, datasets, and applications. With a user-friendly interface and robust tools, we are redefining how machine learning is approached.

---

## Our Offe

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype. See what other students have done in the community-contributions folder -- so many valuable projects -- it's wild!</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 3 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!<br/>
            3. I'm trying out X/Twitter and I'm at <a href="https://x.com/edwarddonner">@edwarddonner<a> and hoping people will teach me how it's done..  
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>